# KDD CUP 2009 - Previsão de relacionamento com o cliente

## Introdução.

A KDD CUP 2009 oferece a oportunidade de trabalhar em grandes bancos de dados de marketing da empresa francesa Telecom, para prever a propensão de clientes a mudar de fornecedor(churn), comprar novos produtos ou serviços(appetency) ou comprar upgrades ou add-ons proposto para eles, tornando a venda mais lucrativa(up-selling).

## Informações fornecidas pela competição:
Dataset usado: **Small**
- Dataset com 50.000 observações
- 230 atributos(variáveis) sendo: 190 numéricos e 40 categóricos.


## Objetivos a serem desenvolvidos:
 - Fazer a limpeza e transformações necessárias
 - Fazer uma análise e exploração dos dados
 - Testar alguns modelos de algoritmos e verificar qual melhor se adapta ao problema

# Imports e configuração do notebook

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import warnings
warnings.filterwarnings('ignore')


# Carga dos dados

In [2]:
df = pd.read_table('dataset/orange_small_train.data')

df['CHURN'] = pd.read_table('dataset/orange_small_train_churn.labels', header=-1)

df['APPETECY'] = pd.read_table('dataset/orange_small_train_appetency.labels', header=-1)

df['UPSELLING'] = pd.read_table('dataset/orange_small_train_upselling.labels', header=-1)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Columns: 233 entries, Var1 to UPSELLING
dtypes: float64(191), int64(4), object(38)
memory usage: 88.9+ MB


In [4]:
df.columns

Index(['Var1', 'Var2', 'Var3', 'Var4', 'Var5', 'Var6', 'Var7', 'Var8', 'Var9',
       'Var10',
       ...
       'Var224', 'Var225', 'Var226', 'Var227', 'Var228', 'Var229', 'Var230',
       'CHURN', 'APPETECY', 'UPSELLING'],
      dtype='object', length=233)



Antes de iniciar as análises podemos verificar que os dados batem com aquilo que foi passado pela competição, não ocorrendo a perda de dados pelo processo de carregamento dos mesmos.
As variáveis estão nomeadas como **Var1** até **Var230**, o que pode indicar que foi feito dessa forma para preservar a privacidade e segurança dos dados sobre os clientes, mas impossibilita o levantamento de algum tipo de hipótese baseado na análise dos nomes das colunas.

# Funções:

Funções adotadas pela análise

In [5]:
# Separação dos dados em dados de treino e teste
#Como a classe dos dados está muito desbalanceada, será adotado a validação cruzada para melhorar o treinamento

def get_train_test(x,y,fold):
    skf = StratifiedKFold(fold)
    for train_index, test_index in skf.split(x, y):
        x_train, x_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]
    
    return x_train, x_test, y_train, y_test


In [6]:
# Treina o modelo e gera a previsão    
def gera_previsao(modelo,x_train,y_train, x_test):
    
    modelo.fit(x_train, y_train)

    return modelo.predict(x_test)

In [7]:
# mostra a informação dos dados

def mostra_info(df, linhas=len(df)):
    for ind, i in enumerate(df.columns[:linhas]):
        print('\n{} \t{} \tnulos: {}\t tipo: {}\t unicos: {}'.format(
        ind,
        i,
        df[i].isnull().sum(),
        df[i].dtype,
        df[i].nunique()
        ))
        print('-' * 100)

In [8]:
# Resumo dos resultados

def gera_resultados(previsao, y_test):
    print(classification_report(y_test, previsao))
    # confusion matrix
    print('\nMATRIZ DE CONFUSÃO\n', confusion_matrix(y_test, previsao))
    print('\nAUC: ', roc_auc_score(y_test, previsao))


# Análise exploratória
Com ela podemos conhecer melhor os dados que estamos trabalhando.

In [9]:
# VISUALIZAR OS PRIMEIROS REGISTROS
df.head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var224,Var225,Var226,Var227,Var228,Var229,Var230,CHURN,APPETECY,UPSELLING
0,NaN,NaN,NaN,NaN,NaN,1526.0,7.0,NaN,NaN,NaN,...,NaN,NaN,xb3V,RAYp,F2FyR07IdsN7I,NaN,NaN,-1,-1,-1
1,NaN,NaN,NaN,NaN,NaN,525.0,0.0,NaN,NaN,NaN,...,NaN,NaN,fKCe,RAYp,F2FyR07IdsN7I,NaN,NaN,1,-1,-1
2,NaN,NaN,NaN,NaN,NaN,5236.0,7.0,NaN,NaN,NaN,...,NaN,kG3k,Qu4f,02N6s8f,ib5G6X1eUxUn6,am7c,NaN,-1,-1,-1
3,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,FSa2,RAYp,F2FyR07IdsN7I,NaN,NaN,-1,-1,-1
4,NaN,NaN,NaN,NaN,NaN,1029.0,7.0,NaN,NaN,NaN,...,NaN,kG3k,FSa2,RAYp,F2FyR07IdsN7I,mj86,NaN,-1,-1,-1


In [10]:
# LINHAS E COLUNAS
df.shape

(50000, 233)

## Informações estatísticas sobre os dados

In [11]:
df.describe(include='all')

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var224,Var225,Var226,Var227,Var228,Var229,Var230,CHURN,APPETECY,UPSELLING
count,702.000000,1241.000000,1240.000000,1579.000000,1.487000e+03,44471.000000,44461.000000,0.0,702.000000,1.487000e+03,...,820,23856,50000,50000,50000,21568,0.0,50000.00000,50000.00000,50000.000000
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,3,23,7,30,4,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4n2X,ELof,FSa2,RAYp,F2FyR07IdsN7I,am7c,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,820,11072,8031,35156,32703,11689,NaN,NaN,NaN,NaN
mean,11.487179,0.004029,425.298387,0.125396,2.387933e+05,1326.437116,6.809496,NaN,48.145299,3.926057e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.85312,-0.96440,-0.852720
std,40.709951,0.141933,4270.193518,1.275481,6.441259e+05,2685.693668,6.326053,NaN,154.777855,9.280896e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.52172,0.26445,0.522373
min,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,NaN,0.000000,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.00000,-1.00000,-1.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000e+00,518.000000,0.000000,NaN,4.000000,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.00000,-1.00000,-1.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000e+00,861.000000,7.000000,NaN,20.000000,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.00000,-1.00000,-1.000000
75%,16.000000,0.000000,0.000000,0.000000,1.187425e+05,1428.000000,7.000000,NaN,46.000000,2.628630e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.00000,-1.00000,-1.000000


Os dados possuem colunas aparentemente com quase sua totalidade constituídas de valores faltantes ou inválidos, precisará ser tratado para a utilização de algoritmos de aprendizado de máquina

## Verificando os valores inválidos

In [12]:
# Porcentagem de valores null
df.isnull().sum() / len(df) * 100

Var1          98.596
Var2          97.518
Var3          97.520
Var4          96.842
Var5          97.026
Var6          11.058
Var7          11.078
Var8         100.000
Var9          98.596
Var10         97.026
Var11         97.520
Var12         98.884
Var13         11.078
Var14         97.520
Var15        100.000
Var16         97.026
Var17         96.842
Var18         96.842
Var19         96.842
Var20        100.000
Var21         11.058
Var22         10.018
Var23         97.026
Var24         14.460
Var25         10.018
Var26         97.026
Var27         97.026
Var28         10.022
Var29         98.596
Var30         98.596
              ...   
Var204         0.000
Var205         3.868
Var206        11.058
Var207         0.000
Var208         0.286
Var209       100.000
Var210         0.000
Var211         0.000
Var212         0.000
Var213        97.742
Var214        50.816
Var215        98.612
Var216         0.000
Var217         1.406
Var218         1.406
Var219        10.422
Var220       

In [13]:
# informações detalhadas
mostra_info(df)


0 	Var1 	nulos: 49298	 tipo: float64	 unicos: 18
----------------------------------------------------------------------------------------------------

1 	Var2 	nulos: 48759	 tipo: float64	 unicos: 2
----------------------------------------------------------------------------------------------------

2 	Var3 	nulos: 48760	 tipo: float64	 unicos: 146
----------------------------------------------------------------------------------------------------

3 	Var4 	nulos: 48421	 tipo: float64	 unicos: 4
----------------------------------------------------------------------------------------------------

4 	Var5 	nulos: 48513	 tipo: float64	 unicos: 571
----------------------------------------------------------------------------------------------------

5 	Var6 	nulos: 5529	 tipo: float64	 unicos: 1486
----------------------------------------------------------------------------------------------------

6 	Var7 	nulos: 5539	 tipo: float64	 unicos: 8
---------------------------------------------

---

Ficou constatado o seguinte:


- A grande maioria do dataset está com valores inválidos
- Seus dados estão codificados, e por isso não é possivel deduzir ou levantar hipóteses sobre os nomes de colunas

# Pré-processamento

## Redução da quantidade de Colunas com dados não significativos

Como o conjunto de dados possui muitos valores inválidos, será mantido apenas aqueles que possuem ao menos 60% de valores válidos.

In [14]:
df = df[[i for i in df.columns if df[i].isnull().mean()<= 0.4]]

In [15]:
# linhas e colunas
df.shape

(50000, 70)

In [16]:
# colunas restantes
df.columns

Index(['Var6', 'Var7', 'Var13', 'Var21', 'Var22', 'Var24', 'Var25', 'Var28',
       'Var35', 'Var38', 'Var44', 'Var57', 'Var65', 'Var73', 'Var74', 'Var76',
       'Var78', 'Var81', 'Var83', 'Var85', 'Var109', 'Var112', 'Var113',
       'Var119', 'Var123', 'Var125', 'Var126', 'Var132', 'Var133', 'Var134',
       'Var140', 'Var143', 'Var144', 'Var149', 'Var153', 'Var160', 'Var163',
       'Var173', 'Var181', 'Var192', 'Var193', 'Var195', 'Var196', 'Var197',
       'Var198', 'Var199', 'Var202', 'Var203', 'Var204', 'Var205', 'Var206',
       'Var207', 'Var208', 'Var210', 'Var211', 'Var212', 'Var216', 'Var217',
       'Var218', 'Var219', 'Var220', 'Var221', 'Var222', 'Var223', 'Var226',
       'Var227', 'Var228', 'CHURN', 'APPETECY', 'UPSELLING'],
      dtype='object')

## Divisão em dados numéricos e categóricos

Assim podemos tratar os dados de acordo com sua necessidade

In [17]:
numericos = df[[i for i in df.columns if(df[i].dtype != 'object')]]
categoricos = df[[i for i in df.columns if (df[i].dtype == 'object')]]

## Tratamento de valores null

- As variáveis numéricas possuiem valores variados, e por isso serão preenchidos por sua média
- As variáveis categóricas terão uma nova categoria com os valores faltantes

In [18]:
for i in numericos.columns[:-3]:
    numericos[i][numericos[i].isnull()] = numericos[i].mean()

categoricos.fillna('faltante', inplace=True)

In [19]:
x = pd.concat((categoricos, numericos.iloc[:, : -3]), axis=1)

## Converte as variáveis categoricas em discretas

Atribui um valor numérico para representar cada categoria diferente

In [20]:
le = LabelEncoder()

In [21]:
for i in x:
    x[i] = le.fit_transform(x[i])

## Padronização dos dados

Deixa os dados em uma mesma escala de valores

In [22]:
scaler = StandardScaler()

x = scaler.fit_transform(x)

# Apresentação dos resultados

---

**EXPLICAÇÃO SOBRE AS MÉTRICAS DE AVALIAÇÃO**

**Classification Report:**  
Mostra um conjunto de métricas, tais como:  
*Precision:* que mostra o número de acertos de cada classe. Como o valor da classe mais assertiva influência o valor da média geral nem sempre esta é uma boa métrica para conjuntos de dados com classes muito desbalanceadas.  
*Recall:* leva em consideração os a quantidade de registros classificados corretamente em ambas as classes.  
*F1 Score:* média entre precisão e recall, ele é melhor utilizado quando o modelo não utiliza probabilidade.

**Confusion Matrix:**  
Com ela é possivel ver a distribuição das classes pelo algoritmo, assim podemos realmente conferir se as previsões não estão tentando seguir algum tipo de tendência.

|Matriz de confusão ||
|-------------------- |--------------- |
|**positivos verdadeiros**|falsos positivos|
|falsos negativos|**negativos verdadeiros**|

**AUC** - Métrica usada pela competição, ela gera uma curva de área formada com as taxas dos valores positivos classificados corretamente e os falsos positivos, ela é usada com conjuntos que estão desbalanceados.\

In [23]:
for i in df.columns[-3:]:
    print('\n\nTARGET - ',i)
    x_train, x_test, y_train, y_test = get_train_test(x,df[i],69)
    previsao = gera_previsao(GaussianNB(), x_train, y_train, x_test)
    gera_resultados(previsao, y_test)
    print('=' * 100)



TARGET -  CHURN
             precision    recall  f1-score   support

         -1       0.95      0.76      0.85       671
          1       0.15      0.55      0.24        53

avg / total       0.90      0.74      0.80       724


MATRIZ DE CONFUSÃO
 [[509 162]
 [ 24  29]]

AUC:  0.6528695554368305


TARGET -  APPETECY
             precision    recall  f1-score   support

         -1       0.99      0.85      0.91       711
          1       0.05      0.42      0.08        12

avg / total       0.97      0.84      0.90       723


MATRIZ DE CONFUSÃO
 [[605 106]
 [  7   5]]

AUC:  0.6337904360056259


TARGET -  UPSELLING
             precision    recall  f1-score   support

         -1       0.95      0.59      0.73       671
          1       0.11      0.62      0.18        53

avg / total       0.89      0.60      0.69       724


MATRIZ DE CONFUSÃO
 [[399 272]
 [ 20  33]]

AUC:  0.6086381913786801
